In [12]:
import pandas as pd
import sqlite3

In [13]:
conn = sqlite3.connect('../data/credit_risk.db')

In [14]:
#Credit risk analysis of portfolio by Grade
query_portfolio = """
SELECT 
    grade,
    COUNT(*) as loan_count,
    SUM(loan_amnt) as total_exposure,
    AVG(int_rate) as avg_interest_rate,
    AVG(is_default) * 100 as default_rate_pct,
    SUM(CASE WHEN is_default = 1 THEN loan_amnt ELSE 0 END) as defaulted_amount,
    SUM(loan_amnt) * AVG(is_default) as expected_loss
FROM loans
GROUP BY grade
ORDER BY grade;
"""

In [15]:
print("Grade-wise protfolo risk:")
portfolio = pd.read_sql(query_portfolio, conn)
portfolio['total_exposure_millions'] = portfolio['total_exposure'] / 1e6
portfolio['expected_loss_millions'] = portfolio['expected_loss'] / 1e6

print(portfolio)

Grade-wise protfolo risk:
  grade  loan_count  total_exposure  avg_interest_rate  default_rate_pct  \
0     A      431435      6291240525           7.084388          3.553258   
1     B      660819      9349769400          10.675230          8.570425   
2     C      646721      9707403875          14.141781         14.241845   
3     D      322455      5058626300          18.136127         20.242514   
4     E      134937      2353300625          21.813907         28.150174   
5     F       41590       794635625          25.435178         36.419812   
6     G       12065       245566350          28.051441         39.958558   

   defaulted_amount  expected_loss  total_exposure_millions  \
0         212916650   2.235440e+08              6291.240525   
1         776361450   8.013150e+08              9349.769400   
2        1370310850   1.382513e+09              9707.403875   
3        1047514375   1.023993e+09              5058.626300   
4         688041525   6.624582e+08              23

In [16]:
print(f"\nTotal Exposure of the Portfolio : ${portfolio['total_exposure'].sum()/1e9:.2f}B")
print(f"Expected total loss: ${portfolio['expected_loss'].sum()/1e6:.1f}M")


Total Exposure of the Portfolio : $33.80B
Expected total loss: $4481.4M


In [17]:
#Information about High Risk customers/borrowers and their characteristics
query_highrisk = """
SELECT 
    CASE 
        WHEN annual_inc < 40000 THEN 'Low Income'
        WHEN annual_inc < 80000 THEN 'Medium Income'
        ELSE 'High Income'
    END as income_bracket,
    CASE
        WHEN dti < 10 THEN 'Low DTI'
        WHEN dti < 20 THEN 'Medium DTI'
        ELSE 'High DTI'
    END as dti_bracket,
    home_ownership,
    COUNT(*) as customer_count,
    AVG(is_default) * 100 as default_rate,
    SUM(loan_amnt) as total_loans
FROM loans
WHERE grade IN ('C', 'D', 'E')
GROUP BY income_bracket, dti_bracket, home_ownership
HAVING customer_count > 1000
ORDER BY default_rate DESC
LIMIT 15;
"""

In [18]:
print("\nProfles of High-Risk Ciustomers with Grades C, D, E:")
profiles_highrisk = pd.read_sql(query_highrisk, conn)
print(profiles_highrisk)


Profles of High-Risk Ciustomers with Grades C, D, E:
   income_bracket dti_bracket home_ownership  customer_count  default_rate  \
0   Medium Income    High DTI           RENT          116087     22.289317   
1      Low Income    High DTI           RENT           62567     21.664775   
2     High Income    High DTI           RENT           33380     20.065908   
3      Low Income  Medium DTI           RENT           37402     19.052457   
4      Low Income    High DTI            OWN           17960     18.997773   
5   Medium Income    High DTI            OWN           31486     18.687671   
6   Medium Income  Medium DTI           RENT           93962     18.591558   
7   Medium Income    High DTI       MORTGAGE          133005     18.130897   
8      Low Income    High DTI       MORTGAGE           28713     18.096333   
9      Low Income  Medium DTI       MORTGAGE           14224     17.575928   
10    High Income  Medium DTI           RENT           48953     17.431005   
11    High

In [19]:
#Monthly emergence and the default trends (Time based trends)
query_trends = """
SELECT 
    SUBSTR(issue_d, -4) || '-' || 
    CASE SUBSTR(issue_d, 1, 3)
        WHEN 'Jan' THEN '01'
        WHEN 'Feb' THEN '02'
        WHEN 'Mar' THEN '03'
        WHEN 'Apr' THEN '04'
        WHEN 'May' THEN '05'
        WHEN 'Jun' THEN '06'
        WHEN 'Jul' THEN '07'
        WHEN 'Aug' THEN '08'
        WHEN 'Sep' THEN '09'
        WHEN 'Oct' THEN '10'
        WHEN 'Nov' THEN '11'
        WHEN 'Dec' THEN '12'
    END as year_month,
    COUNT(*) as loans_originated,
    SUM(loan_amnt) as total_amount,
    AVG(is_default) * 100 as default_rate,
    AVG(int_rate) as avg_interest_rate
FROM loans
WHERE issue_d IS NOT NULL
GROUP BY year_month
ORDER BY year_month;
"""

In [20]:
print("\nMonthly Emergence Trends:")
trends_m = pd.read_sql(query_trends, conn)
print(trends_m.head(20))


Monthly Emergence Trends:
   year_month  loans_originated  total_amount  default_rate  avg_interest_rate
0     2007-06                23         85350     13.043478           9.876957
1     2007-07                62        341625     11.290323          11.213548
2     2007-08                72        503300     27.777778          11.653333
3     2007-09                53        372950     24.528302          12.463208
4     2007-10               105        753225     32.380952          12.438476
5     2007-11               112       1008650     30.357143          11.962321
6     2007-12               172       1887175     27.325581          11.810523
7     2008-01               305       2926000     27.540984          11.720393
8     2008-02               306       2959225     19.934641          12.195915
9     2008-03               401       4141600     22.194514          12.305810
10    2008-04               259       2433875     24.324324          12.381004
11    2008-05            

In [21]:
print(f"\nTotal number months analyzed: {len(trends_m)}")
print(f"\nHighest default rate month:")
print(trends_m.nlargest(1, 'default_rate'))


Total number months analyzed: 139

Highest default rate month:
  year_month  loans_originated  total_amount  default_rate  avg_interest_rate
4    2007-10               105        753225     32.380952          12.438476


In [22]:
print(f"\nMonth with the Lowest Default Rate:")
print(trends_m.nsmallest(1, 'default_rate'))


Month with the Lowest Default Rate:
    year_month  loans_originated  total_amount  default_rate  \
138    2018-12             39662     628212000      0.284907   

     avg_interest_rate  
138          12.965485  


In [23]:
#loan default rates by age
query_vintage = """
SELECT 
    term,
    SUBSTR(issue_d, -4) as year,
    COUNT(*) as loan_count,
    AVG(is_default) * 100 as default_rate,
    SUM(loan_amnt) / 1000000.0 as total_volume_millions
FROM loans
WHERE SUBSTR(issue_d, -4) IN ('2013', '2014', '2015')
GROUP BY term, year
ORDER BY year, term;
"""

In [24]:
print("\nVintage Analysis for 2013-2015 loans by term:")
vintage = pd.read_sql(query_vintage, conn)
print(vintage)


Vintage Analysis for 2013-2015 loans by term:
   term  year  loan_count  default_rate  total_volume_millions
0    36  2013      100414     12.326966            1271.895225
1    60  2013       34390     25.164292             710.603800
2    36  2014      162541     13.726998            2045.333375
3    60  2014       73054     26.244969            1457.662925
4    36  2015      283090     14.913632            3624.847950
5    60  2015      137874     25.271625            2790.009850


In [26]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('../data/credit_risk.db')

# Exporting sample for Tableau which is 100,000 loans for performance purposes
# Stratified the sample to maintain the grade distribution
df_tableau = pd.read_sql("""
SELECT * FROM loans 
ORDER BY RANDOM() 
LIMIT 100000
""", conn)

df_tableau.to_csv('../data/processed/loans_tableau.csv', index=False)

print(f"Exported {len(df_tableau):,} loans for Tableau")
print(f"File saved: data/processed/loans_tableau.csv")
print(f"\nDefault rate in sample: {df_tableau['is_default'].mean()*100:.2f}%")
print(f"Grades distribution:")
print(df_tableau['grade'].value_counts().sort_index())

conn.close()

Exported 100,000 loans for Tableau
File saved: data/processed/loans_tableau.csv

Default rate in sample: 12.66%
Grades distribution:
grade
A    19172
B    29437
C    28682
D    14347
E     6013
F     1820
G      529
Name: count, dtype: int64
